In [1]:
import pennylane as qml # type: ignore
from pennylane import numpy as np # type: ignore

In [2]:
import warnings
warnings.filterwarnings("ignore")

import torch # type: ignore

from botorch.models import SingleTaskGP # type: ignore
from botorch.fit import fit_gpytorch_mll # type: ignore
from botorch.utils.transforms import normalize, unnormalize # type: ignore
from gpytorch.mlls import ExactMarginalLogLikelihood # type: ignore
from botorch.acquisition import LogExpectedImprovement # type: ignore
from botorch.optim import optimize_acqf # type: ignore

In [3]:
symbols = ['O', 'H', 'H', 'H']
coords = np.array([
    [ 0.000000,  0.000000,  0.128500],  # Oxygen (Top of pyramid)
    [ 0.000000,  0.937600, -0.165400],  # H1
    [ 0.812000, -0.468800, -0.165400],  # H2
    [-0.812000, -0.468800, -0.165400]   # H3
], requires_grad=False)

molecule = qml.qchem.Molecule(
    symbols,
    coords,
    charge=1,
    mult=1,
    basis_name='6-31G',
    name='hydronium',
    unit='angstrom'
)

H, qubits = qml.qchem.molecular_hamiltonian(
    molecule,
    mapping='jordan_wigner',
    method='pyscf',
    active_electrons=8,
    active_orbitals=8
)

coeffs, obs = H.terms()

print(f"Using {qubits} qubits to encode the hydronium ion.")
print(f"Molecular Hamiltonian has {len(obs)} operators.")



Using 16 qubits to encode the hydronium ion.
Molecular Hamiltonian has 3053 operators.


In [4]:
hf_energy = qml.qchem.hf_energy(molecule)
hf_energy()

array(-76.26601288)

In [5]:
H_sparse = qml.SparseHamiltonian(H.sparse_matrix(), wires=range(qubits))
dev = qml.device("lightning.qubit", wires=qubits)

In [6]:
hf_state = qml.qchem.hf_state(8, qubits)
print(f"Hartree-Fock state: {hf_state}")

Hartree-Fock state: [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]


In [7]:
def givens_network(params, qubits, layers=2):
    """
    Constructs a Brick-Wall Givens Rotation Network.
    
    Args:
        params: Flat list of parameters from the optimizer
        wires: List of wire indices (e.g., range(16))
        layers: How many "brick rows" to build.
    """    
    param_idx = 0
    for layer in range(layers):
        # Even-Odd pairs (0-1, 2-3...)
        for i in range(0, qubits - 1, 2):
            if param_idx < len(params):
                qml.SingleExcitation(params[param_idx], wires=[i, i+1])
                param_idx += 1
        
        # Odd-Even pairs (1-2, 3-4...)
        # We skip the first and last wire to create the "offset"
        for i in range(1, qubits - 1, 2):
            if param_idx < len(params):
                qml.SingleExcitation(params[param_idx], wires=[i, i+1])
                param_idx += 1

In [15]:
@qml.qnode(dev)
def cost_fn(params, layers=1):
    qml.BasisState(hf_state, wires=range(qubits))
    givens_network(params, qubits, layers)
    return qml.expval(H_sparse)

In [16]:
def objective_function(parameters):
    """
    Args: parameters (Tensor): shape (1, n_layers *n_params)
    Returns: energy (Tensor): shape (1, 1)
    """
    x_np = parameters.detach().numpy().flatten()
    energy = cost_fn(x_np)
    return -torch.tensor([energy], dtype=torch.double).unsqueeze(0) # minimization

In [17]:
objective_function(torch.zeros(1, 2 * (qubits // 2 + (qubits - 1) // 2), dtype=torch.double))

tensor([[76.2660]], dtype=torch.float64)

In [18]:
n_layers = 2
n_params = (qubits // 2 + (qubits - 1) // 2)
bounds = torch.tensor([[-np.pi] * n_params * n_layers, [np.pi] * n_params * n_layers], dtype=torch.double)
print(f"Total parameters: {n_layers * n_params}")

Total parameters: 30


In [19]:
n_iterations = 50
n_initial_points = 10

In [22]:
# 1. Generate Initial Data (Random initialization)
train_x = torch.normal(mean=0, std=0.1, size=(n_initial_points, n_params * n_layers), dtype=torch.double, requires_grad=True)

# 3. Calculate Energies for all 10 points
# We loop because 'objective_function' expects (1, D), not (N, D)
train_obj = torch.cat([
    objective_function(x.unsqueeze(0)) for x in train_x
])

print(f"Shapes: X={train_x.shape}, Y={train_obj.shape}")

Shapes: X=torch.Size([10, 30]), Y=torch.Size([10, 1])


In [23]:
print(f"--- Starting Optimization (30 Params, {n_iterations} Steps) ---")

best_energies = [-train_obj.max().item()]
for i in range(n_iterations):
    # Fit the Gaussian Process
    # The model learns the shape of the energy landscape
    model = SingleTaskGP(train_x, train_obj)
    mll = ExactMarginalLogLikelihood(model.likelihood, model)
    fit_gpytorch_mll(mll)
    # Optimize the Acquisition Function
    # "Where should we look next?" (Balancing exploration vs exploitation)
    acq_func = LogExpectedImprovement(model=model, best_f=train_obj.max())
    
    candidate, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=1,              
        num_restarts=8,  
        raw_samples=128,
    )
    
    new_val = objective_function(candidate)
    train_x = torch.cat([train_x, candidate])
    train_obj = torch.cat([train_obj, new_val])
    current_best = -train_obj.max().item()
    best_energies.append(current_best)
    print(f"Step {i+1:02d} | New: {-new_val.item():.5f} Ha | Best: {current_best:.5f} Ha")

print(f"\n--- optimization Finished ---")
print(f"Final VQE Energy: {current_best:.6f} Ha")

--- Starting Optimization (30 Params, 50 Steps) ---
Step 01 | New: -76.26559 Ha | Best: -76.26601 Ha
Step 02 | New: -76.26578 Ha | Best: -76.26601 Ha
Step 03 | New: -76.15859 Ha | Best: -76.26601 Ha
Step 04 | New: -76.23848 Ha | Best: -76.26601 Ha
Step 05 | New: -76.26595 Ha | Best: -76.26601 Ha
Step 06 | New: -76.26592 Ha | Best: -76.26601 Ha
Step 07 | New: -76.26598 Ha | Best: -76.26601 Ha
Step 08 | New: -76.26579 Ha | Best: -76.26601 Ha
Step 09 | New: -76.26593 Ha | Best: -76.26601 Ha
Step 10 | New: -76.26597 Ha | Best: -76.26601 Ha
Step 11 | New: -76.26598 Ha | Best: -76.26601 Ha
Step 12 | New: -76.26601 Ha | Best: -76.26601 Ha
Step 13 | New: -76.26599 Ha | Best: -76.26601 Ha
Step 14 | New: -76.26601 Ha | Best: -76.26601 Ha
Step 15 | New: -76.26601 Ha | Best: -76.26601 Ha
Step 16 | New: -76.26600 Ha | Best: -76.26601 Ha
Step 17 | New: -76.26601 Ha | Best: -76.26601 Ha
Step 18 | New: -76.26601 Ha | Best: -76.26601 Ha
Step 19 | New: -76.26601 Ha | Best: -76.26601 Ha
Step 20 | New: -7